In [1]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import UnstructuredHTMLLoader 
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.llms import HuggingFaceHub
from langchain.chains import LLMChain, ConversationChain
from langchain.prompts import PromptTemplate
from langchain.memory import ChatMessageHistory, ConversationBufferMemory,ConversationSummaryMemory

from langchain_community.llms import HuggingFaceEndpoint





In [2]:
import json

# Öffne die JSON-Datei und lade den Inhalt
with open('/Users/riccardo/Desktop/Repositorys_Github/LLM/Docs/api_token.json', 'r') as api_file:
    api_token_file = json.load(api_file)

# Extrahiere die Variable aus den Daten
api_token = api_token_file['Hugging_face_token']

In [3]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50, 
    length_function = len)
text_splitter

In [4]:
filepath = '/Users/riccardo/Desktop/Repositorys_Github/LLM/Docs/doc_1.pdf'
loader = PyPDFLoader(filepath)
chunks = loader.load_and_split(text_splitter=text_splitter)

In [5]:
for chunk in chunks[:5]:
    print("Page content: \n", chunk.page_content),
    print("Page_metadata: \n", chunk.metadata),
    print("----------------------------")

Page content: 
 Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.comNoam Shazeer∗
Google Brain
noam@google.comNiki Parmar∗
Google Research
nikip@google.comJakob Uszkoreit∗
Google Research
usz@google.com
Llion Jones∗
Google Research
llion@google.comAidan N. Gomez∗ †
University of Toronto
aidan@cs.toronto.eduŁukasz Kaiser∗
Google Brain
lukaszkaiser@google.com
Illia Polosukhin∗ ‡
illia.polosukhin@gmail.com
Abstract
The dominant sequence transduction models are based on complex recurrent or
convolutional neural networks that include an encoder and a decoder. The best
performing models also connect the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
Pa

In [6]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

/Users/riccardo/anaconda3/envs/llm_train/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
embedding = embedding_function.embed_documents("This is a test sentence.")

print(embedding[0])
print("Dimension of Embedding: ", len(embedding[0]))

[-0.05895749852061272, 0.05387335643172264, 0.008316715247929096, 0.0206805057823658, 0.0022009313106536865, -0.034434445202350616, 0.08473188430070877, 0.1077682226896286, 0.058957234025001526, -0.016638653352856636, 0.06211813539266586, -0.0740492045879364, -0.0303032286465168, 0.03807591274380684, -0.004251338075846434, -0.007986415177583694, 0.0031452886760234833, -0.07220274955034256, -0.01501194667071104, -0.07136189937591553, -0.08817413449287415, 0.0452018640935421, 0.009071135893464088, 0.041956428438425064, -0.0012928870273754, -0.014840386807918549, -0.03942627087235451, 0.007077477872371674, -0.036413680762052536, -0.042899180203676224, -0.04982227832078934, 0.009418361820280552, -0.022699205204844475, 0.010238043032586575, -0.00445396825671196, -0.07801611721515656, -0.070695661008358, -0.005806431174278259, 0.03875807300209999, 0.01204211637377739, -0.05325385555624962, -0.0963793396949768, -0.028097203001379967, -0.009883742779493332, 0.06731418520212173, 0.0042782085947

In [8]:
db = Chroma.from_documents(chunks, embedding_function)

In [9]:
print("Chunks in DB:", db._collection.count())

Chunks in DB: 48


In [10]:
query = "Write a summary of the first page of the document."
retriever = db.as_retriever()
retriever.get_relevant_documents(query)

[Document(page_content='[25] Mitchell P Marcus, Mary Ann Marcinkiewicz, and Beatrice Santorini. Building a large annotated\ncorpus of english: The penn treebank. Computational linguistics , 19(2):313–330, 1993.\n[26] David McClosky, Eugene Charniak, and Mark Johnson. Effective self-training for parsing. In\nProceedings of the Human Language Technology Conference of the NAACL, Main Conference ,\npages 152–159. ACL, June 2006.\n[27] Ankur Parikh, Oscar Täckström, Dipanjan Das, and Jakob Uszkoreit. A decomposable attention\nmodel. In Empirical Methods in Natural Language Processing , 2016.\n[28] Romain Paulus, Caiming Xiong, and Richard Socher. A deep reinforced model for abstractive\nsummarization. arXiv preprint arXiv:1705.04304 , 2017.\n[29] Slav Petrov, Leon Barrett, Romain Thibaux, and Dan Klein. Learning accurate, compact,\nand interpretable tree annotation. In Proceedings of the 21st International Conference on\nComputational Linguistics and 44th Annual Meeting of the ACL , pages 4

In [11]:
llm = HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', 
                     huggingfacehub_api_token=api_token,  
                     model_kwargs={"max_length": 100})


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/riccardo/.cache/huggingface/token
Login successful


In [12]:
qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm = llm,
    chain_type = "stuff",  
    retriever = retriever
    )

qa_with_sources

RetrievalQAWithSourcesChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try to make up an answer.\nALWAYS return a "SOURCES" part in your answer.\n\nQUESTION: Which state/country\'s law governs the interpretation of the contract?\n=========\nContent: This Agreement is governed by English law and the parties submit to the exclusive jurisdiction of the English courts in  relation to any dispute (contractual or non-contractual) concerning this Agreement save that either party may apply to any court for an  injunction or other relief to protect its Intellectual Property Rights.\nSource: 28-pl\nContent: No Waiver. Failure or delay in exercising any right or remedy under this Agreement shall not constitute a w

In [14]:
query = "How are the writer of the document."
qa_with_sources.invoke(query)

{'question': 'How are the writer of the document.',
 'answer': ' The writers of the document are Mitchell P Marcus, Mary Ann Marcinkiewicz, Beatrice Santorini, David McClosky, Eugene Charniak, Mark Johnson, Ankur Parikh, Oscar Täckström, Dipanjan Das, Jakob Uszkoreit, Romain Paulus, Caiming Xiong, Richard Socher, Slav Petrov, Leon Barrett, Romain Thibaux, Dan Klein.\n',
 'sources': '/Users/riccardo/Desktop/Repositorys_Github/LLM/Docs/doc_1.pdf'}

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


In [ ]:
question = "Who won the FIFA World Cup in the year 1994? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate.from_template(template)

In [ ]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

llm = HuggingFaceEndpoint(
    repo_id=repo_id, temperature=0.5, huggingfacehub_api_token=api_token, 
    model_kwargs={"max_length": 100}
)


llm_chain = LLMChain(prompt=prompt, llm=llm)


print(llm_chain.run(question))

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50, 
    length_function = len)


filepath = '/Users/riccardo/Desktop/Repositorys_Github/LLM/Docs/storytelling-with-data-cole-nussbaumer-knaflic.pdf'
loader = PyPDFLoader(filepath)
chunks = loader.load_and_split(text_splitter=text_splitter)

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

embedding = embedding_function.embed_documents("This is a test sentence.")
db = Chroma.from_documents(chunks, embedding_function)
query = "Write a summary of the first page of the document."
retriever = db.as_retriever()
retriever.get_relevant_documents(query)
llm = HuggingFaceHub(repo_id='mistralai/Mixtral-8x7B-Instruct-v0.1', 
                     huggingfacehub_api_token=api_token,  
                     model_kwargs={"max_length": 100})
qa_with_sources = RetrievalQAWithSourcesChain.from_chain_type(
    llm = llm,
    chain_type = "stuff",  
    retriever = retriever
    )



In [ ]:
query = "Give me the title of the book."
print(qa_with_sources.invoke(query))